<a href="https://colab.research.google.com/github/daniel-hain/paper_skill_development_NO/blob/main/10_embeddings_sbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies needed for this notebook

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [33]:
!pip install autofaiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 21.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115941 sha256=9d972759611f3549b87a43e073f4cea8b3d26f5002847ba25ed0e2777cfd2939
  Stored in directory: /root/.cache/pip/wheels/2a/93/86/8cd17bc6c40fb605c3ac549d0b860ef7e84ee5f67bf01a3287
Successfully built fire
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0


In [1]:
!pip install -qU transformers sentence-transformers

In [22]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
import torch

from autofaiss import build_index

In [23]:
import pickle
import pandas as pd
import numpy as np

---

## Using sentence-transformers

In [6]:
model = SentenceTransformer('NbAiLab/nb-sbert-base')

# Load data

In [7]:
# Read the CSV file
df_skills = pd.read_csv("/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/text_skills_plain.csv")
df_jobs = pd.read_csv("/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/text_job_plain.csv")

# Embeddings Skills

In [8]:
list_skills = df_skills['text']
list_skills = list_skills.tolist()

In [9]:
skills_embeddings = model.encode(list_skills, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/435 [00:00<?, ?it/s]

In [25]:
np.save("/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/skill_embeddings.npy", skills_embeddings.cpu() ) 

In [27]:
# Build a KNN index
!autofaiss build_index --embeddings="/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/" \
                    --index_path="knn.index" \
                    --index_infos_path="infos.json" \
                    --metric_type={metric_type} \
                    --max_index_query_time_ms=5 \
                    --max_index_memory_usage={max_index_memory_usage}

Traceback (most recent call last):
  File "/usr/local/bin/autofaiss", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.9/dist-packages/autofaiss/external/quantize.py", line 606, in main
    fire.Fire(
  File "/usr/local/lib/python3.9/dist-packages/fire/core.py", line 141, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/usr/local/lib/python3.9/dist-packages/fire/core.py", line 466, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/usr/local/lib/python3.9/dist-packages/fire/core.py", line 681, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/autofaiss/external/quantize.py", line 181, in build_index
    max_index_bytes = cast_memory_to_bytes(max_index_memory_usage)
  File "/usr/local/lib/python3.9/dist-packages/autofaiss/utils/cast.py", line 31, in cast_memory_to_bytes
    matching_groups = re.findall(number_match + unit_match, memory_strin

# Semantic search

In [28]:
# Function that searches the corpus and prints the results
def sem_search(input_text):
    input_embedding = model.encode(input_text, convert_to_tensor=True)
    hits = util.semantic_search(input_embedding, skills_embeddings)
    hits = hits[0]  #Get the hits for the first query

    print("Input ", input_text)
    for hit in hits[0:10]:
        print("\t{:.3f}\t{}".format(hit['score'], list_skills[hit['corpus_id']]))

In [29]:
test_sentence = 'Teoriene om kunstig intelligens, anvendte prinsipper, arkitekturer og systemer, f.eks. intelligente agenter, multiagentsystemer, ekspertsystemer, regelbaserte systemer, nevrale nettverk, ontologier og kognisjonsteorier.'

In [18]:
test_sentence = df_jobs.iloc[30,1]
test_sentence

'dokumentasjon rapportering helsefaglærlinger arbeider enten hjemmetjeneste institusjon to år kvalifikasjoner bestått vg helse sosialfag vg helsefag personlige egenskaper vektlegger vårt personale evne samarbeid'

In [20]:
test_sentence1 = "Som vår nye data scientist vil du delta i tverrfaglige team som utvikler, implementerer og drifter maskinlæringsmodeller og regelbaserte modeller. "

In [31]:
test_sentence2 = "Videreutvikle og tilpasse infrastruktur for diagnostiske og behandlingsbiobanker for kvalitetshevning av håndtering, oppbevaring, og tilgjengeliggjøring av pasientmateriale"

In [30]:
sem_search(test_sentence1)

Input  Som vår nye data scientist vil du delta i tverrfaglige team som utvikler, implementerer og drifter maskinlæringsmodeller og regelbaserte modeller. 
	0.725	Delta i de ulike utviklingsfasene i dataprogrammer for  økonometrisk og statistisk analyse, som forskning, utvikling av nye produkter, utforming av prototyper og vedlikehold.
	0.708	Bruke digitale verktøy og teknologier for å skape kunnskap og for å innovere prosesser og produkter. Delta i kognitiv behandling enkeltvis og samlet for å forstå og løse konseptuelle problemer og problemsituasjoner i digitale miljøer.
	0.707	Utvikle en programvare som styrer en IKT-enhet og dens samspill med andre applikasjoner.
	0.692	De metoder og verktøy som kreves for å utvikle systemprogramvare, spesifikasjoner for systemarkitektur og grensesnitt mellom nettverk, systemmoduler og komponenter.
	0.688	Den tverrfaglige grenen av ingeniørarbeid som håndterer dynamiske systemer med informasjon om hvordan deres atferd endres ved tilbakemeldinger.
	0

In [14]:
# Save
with open('/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/skill_embeddings.pickle', 'wb') as pkl:
    pickle.dump(skills_embeddings, pkl)

#with open('skill_embedding.pickle', 'rb') as pkl:
 #   doc_embedding = pickle.load(pkl)

# Embedding jobs

In [15]:
list_jobs = df_jobs['text']
list_jobs = list_jobs.tolist()

In [16]:
jobs_embeddings = model.encode(list_jobs, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/5144 [00:00<?, ?it/s]

In [17]:
# Save
with open('/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/job_embeddings.pickle', 'wb') as pkl:
    pickle.dump(jobs_embeddings, pkl)

#with open('skill_embedding.pickle', 'rb') as pkl:
 #   doc_embedding = pickle.load(pkl)